## 보스턴 주택 가격 예측: 딥러닝을 이용한 회귀 분석

이 노트북은 Keras를 사용하여 보스턴 주택 가격 데이터셋으로 회귀 모델을 구축, 훈련 및 평가하는 과정을 다룹니다. 딥러닝이 분류 문제뿐만 아니라 수치 예측(회귀) 문제에도 어떻게 적용될 수 있는지 보여줍니다.

### 1. 라이브러리 임포트 및 데이터 로드
필요한 라이브러리를 임포트하고 Keras에 내장된 보스턴 주택 가격 데이터셋을 로드합니다. 데이터의 형태와 처음 몇 개의 샘플을 출력하여 구조를 확인합니다.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import boston_housing

(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

print("--- 데이터 형태 ---")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

print("\n--- 데이터 샘플 ---")
print("X_train first 5 samples:\n", X_train[:5])
print("\ny_train first 5 samples:\n", y_train[:5])

### 2. 데이터 전처리: 특성 스케일링
입력 특성(X)들은 서로 다른 범위의 값을 가집니다. 모델이 안정적으로 학습하기 위해 모든 특성들이 비슷한 범위를 갖도록 스케일링을 진행합니다. 여기서는 `Normalizer`를 사용하여 각 샘플(행)의 유클리드 노름(L2 norm)이 1이 되도록 조정합니다.

In [ ]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
X_train_scaled = normalizer.fit_transform(X_train)
X_test_scaled = normalizer.transform(X_test) # 테스트 데이터는 훈련 데이터 기준으로 변환

print("--- 스케일링 후 데이터 샘플 ---")
print("X_train_scaled first 5 samples:\n", X_train_scaled[:5])

### 3. 딥러닝 모델 정의
회귀 문제를 풀기 위한 `Sequential` 모델을 정의합니다.

- **입력층**: 13개의 특성을 받습니다 (`input_shape=(13,)`).
- **은닉층**: `relu` 활성화 함수를 사용하는 여러 개의 `Dense` 층으로 구성하여 복잡한 비선형 관계를 학습합니다.
- **출력층**: 단일 주택 가격을 예측해야 하므로, 활성화 함수가 없는 하나의 `Dense` 층으로 구성합니다. 이를 통해 어떤 값이든 자유롭게 예측할 수 있습니다.

모델 컴파일 시:
- **Optimizer**: `rmsprop`을 사용합니다.
- **Loss**: 회귀 문제의 대표적인 손실 함수인 `mse` (Mean Squared Error, 평균 제곱 오차)를 사용합니다.
- **Metrics**: `mae` (Mean Absolute Error, 평균 절대 오차)를 추가하여 훈련 과정을 모니터링합니다. MAE는 예측 오차를 직관적으로 이해하는 데 도움이 됩니다.

In [ ]:
from tensorflow.keras import models, layers

def makeModel():
    model = models.Sequential([
        layers.Dense(512, activation='relu', input_shape=(13,)),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        # 출력층: 회귀는 특정 값을 예측하므로 활성화 함수 없이 단일 뉴런으로 구성
        layers.Dense(1)
    ])

    model.compile(optimizer='rmsprop', 
                  loss='mse', 
                  metrics=['mae'])
    return model

network = makeModel()
network.summary()

### 4. 모델 훈련
`fit()` 메서드를 사용하여 스케일링된 훈련 데이터로 모델을 학습시킵니다.

In [ ]:
history = network.fit(X_train_scaled, y_train, epochs=10, batch_size=100, validation_split=0.2, verbose=1)

### 5. 모델 평가
`evaluate()` 메서드를 사용하여 훈련된 모델의 성능을 훈련 데이터셋과 테스트 데이터셋에 대해 각각 평가합니다. 손실(MSE)과 평균 절대 오차(MAE)를 출력합니다.

In [ ]:
print("\n--- 모델 평가 ---")
train_loss, train_mae = network.evaluate(X_train_scaled, y_train)
test_loss, test_mae = network.evaluate(X_test_scaled, y_test)

print(f"훈련셋   => 손실(MSE): {train_loss:.2f}, MAE: {train_mae:.2f}")
print(f"테스트셋 => 손실(MSE): {test_loss:.2f}, MAE: {test_mae:.2f}")